In [1]:
import os
import pandas as pd
import numpy as np

import sklearn as sk

from lightgbm import LGBMClassifier

#データの読み込み
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')


In [2]:
train.info()
train.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:

train = train.drop(['Name', 'Embarked', 'Ticket'], axis = 'columns')
train = train.fillna(train['Age'].median())
train['Sex'] = (train['Sex'] == 'female') * 1
train['FamilySize'] = train['SibSp'] + train['Parch'] + 1
train = train.drop(['SibSp', 'Parch'], axis ='columns')
train['Cabin'] = train['Cabin'].apply(lambda x:0 if type(x) == float else 1)


In [4]:
train.info()
train.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 8 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Sex            891 non-null int64
Age            891 non-null float64
Fare           891 non-null float64
Cabin          891 non-null int64
FamilySize     891 non-null int64
dtypes: float64(2), int64(6)
memory usage: 55.8 KB


,PassengerId,Survived,Pclass,Sex,Age,Fare,Cabin,FamilySize
0,1,0,3,0,22.0,7.2500,0,2
1,2,1,1,1,38.0,71.2833,1,2
2,3,1,3,1,26.0,7.9250,0,1
3,4,1,1,1,35.0,53.1000,1,2
4,5,0,3,0,35.0,8.0500,0,1


In [5]:
train_data = train.values
xs = train_data[:, 2:]
y = train_data[:, 1]

print (xs.shape)
print (y.shape)

(891, 6)
(891,)


In [6]:
#テストデータにもトレーニングデータと同じ処理を行う
test = test.drop(['Name', 'Embarked', 'Ticket'], axis = 'columns')
test = test.fillna(train['Age'].median())
test['Sex'] = (test['Sex'] == 'female') * 1

test['FamilySize'] = test['SibSp'] + test['Parch'] + 1
test = test.drop(['SibSp', 'Parch'], axis ='columns')

test['Cabin'] = test['Cabin'].apply(lambda x:0 if type(x) == float else 1)


test_data = test.values
xs_test = test_data[:, 1:]

print(xs_test.shape)

(418, 6)


In [7]:
from lightgbm import LGBMClassifier
model = LGBMClassifier(learning_rate=0.01,max_depth=2,num_leaves=3).fit(xs,y)
output = model.predict(xs_test).astype(int)
print(output)

[0 1 0 0 1 0 1 0 1 0 0 0 1 0 1 1 0 0 1 1 0 0 1 0 1 0 1 0 0 0 0 0 1 1 0 0 1
 1 0 0 0 0 0 1 1 0 0 0 1 1 0 0 1 1 0 0 0 0 0 1 0 0 0 1 0 1 1 0 0 1 1 0 1 0
 1 0 0 1 0 1 0 0 0 0 0 0 1 1 1 0 1 0 1 0 0 0 1 0 1 0 1 0 0 0 1 0 0 0 0 0 0
 1 1 1 1 0 0 1 0 1 1 0 1 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0
 0 0 1 0 0 1 0 0 1 1 0 1 1 0 1 0 0 1 0 0 1 1 0 0 0 0 0 1 1 0 1 1 0 0 1 0 1
 0 1 0 0 0 0 0 0 0 0 0 0 1 0 1 1 0 0 1 0 0 1 0 1 0 0 0 0 1 0 0 1 0 1 0 1 0
 1 0 1 1 0 1 0 0 0 1 0 0 0 0 0 0 1 1 1 1 0 0 0 0 1 0 1 1 1 0 0 0 0 0 0 0 1
 0 0 0 1 1 0 0 0 0 1 0 0 0 1 1 0 1 0 0 0 0 1 0 1 1 1 0 0 0 0 0 0 1 0 0 0 0
 1 0 0 0 0 0 0 0 1 1 0 0 0 1 0 0 0 1 1 1 0 0 0 0 0 0 0 0 1 0 1 0 0 0 1 0 0
 1 0 0 0 0 0 0 0 0 0 1 0 1 0 1 0 1 1 0 0 0 1 0 1 0 0 1 0 1 1 0 1 0 0 1 1 0
 0 1 0 0 1 1 1 0 0 0 0 0 1 1 0 1 0 0 0 0 0 1 0 0 0 1 0 1 0 0 1 0 1 0 0 0 0
 0 1 1 1 1 1 0 1 0 0 0]


In [8]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 7 columns):
PassengerId    418 non-null int64
Pclass         418 non-null int64
Sex            418 non-null int64
Age            418 non-null float64
Fare           418 non-null float64
Cabin          418 non-null int64
FamilySize     418 non-null int64
dtypes: float64(2), int64(5)
memory usage: 23.0 KB


In [9]:
test['Survived'] = output
test.to_csv('output.csv', columns = ['PassengerId', 'Survived'], index = False)

In [10]:
test['Survived'] = output
test.to_csv('output.csv', columns = ['PassengerId', 'Survived'], index = False)

In [11]:
#モデルの性能評価
model.score(xs, y)
#交差検証
from sklearn.model_selection import cross_val_score
scores = cross_val_score(model, xs, y, cv=5, scoring = 'neg_mean_squared_error')

In [12]:
rmse_scores = np.sqrt(-scores)
print(rmse_scores)
print(rmse_scores.mean())
print(rmse_scores.std())


[0.5013947  0.46808239 0.43704832 0.49718308 0.43057317]
0.46685633305924695
0.029390810368355513
